In [11]:
import bs4 as bs
import requests

import pandas as pd

import os

import time
import random

import sqlalchemy as sal
from sqlalchemy import create_engine
from sqlalchemy.sql import text

import duckdb

In [12]:
# union of all season and team to one table for each table
servername = '****'
football_data = 'Football_Data'
trusted_conneciton = '?trusted_conneciton=yes'
driver = '&driver=ODBC+Driver+17+for+SQL+Server'
username = 'MAKeith92'
password = 'Lopez!123'

In [13]:
#create engine url to connect to sql server
engine = create_engine(f'mssql+pyodbc://{servername}/{football_data}{trusted_conneciton}{driver}')
engine

Engine(mssql+pyodbc://DESKTOP-5IAPFQC\MSSQLSERVER2019/Football_Data?driver=ODBC+Driver+17+for+SQL+Server&trusted_conneciton=yes)

In [14]:
conn = engine.connect()

In [15]:
years = range(2010,2024)

In [16]:
weeks = range(1,18)

In [17]:
position_li = ['qb', 'rb', 'wr', 'te', 'k', 'dst', 'dl', 'lb', 'db']

In [5]:
def clean_df(df, year, week, position):
    try:
        df = df.droplevel(0, axis=1)

    except:
        pass

    df['Player'] = df['Player'].apply(lambda x: x.split(' ')[:2])
    df['Player'] = df['Player'].apply(lambda x: ' '.join(x))

    #Add Position Column
    df['Position'] = [position.upper()] * len(df)

    #add Year Column
    df['Year'] = [year] * len(df)

    df['Week'] = [week] * len(df)

    return df

In [6]:
def qb_col(df):
    #quarterback columns
    df.columns = ['Rank', 'Player', 'Pass Completions', 'Pass Attempts',
                  'Pass Completion/Attempts', 'Passing Yards', 'Passsing Yards/Attempt', 'Passing Touchdowns', 
                  'Interception', 'Sacks', 'Rushing Attempts', 'Rushing Yards', 
                  'Rushing Touchdown', 'Fumbles', 'Games Played', 'Fantasy Football Points', 
                  'FTPS/Game', 'ROST', 'Position', 'Year']
    return df

In [7]:
def rb_col(df):
    #Running Back
    df.columns = ['Rank', 'Player', 'Rushing Attempts', 'Rushing Yards', 'Rushing Yards/Attempts',
                  'Longest Rushing', 'Rushing 20+', 'Rushing Touchdown',
                  'Recptions', 'TGT', 'Receiving Yards', 'Receiving Yards/Reception',
                  'Receiving Touchdowns', 'Fumbles', 'Games Played', 'Fantasy Football Points', 'FTPS/Game', 'ROST',
                  'Position', 'Year']
    return df

In [8]:
def wr_col(df):
    #wide receiver
    df.columns = ['Rank', 'Player', 'Receiving Receptions', 'Receiving Target', 'Receiving Yards',
                  'Yards/Reception', 'Longest Reception', 'Receiving 20+',
                  'Receiving Touchdowns', 'Rushing Attempts', 'Rushing Yards',
                  'Rushing Touchdown', 'Fumbles', 'Games Played', 'Fantasy Football Points', 'FTPS/Game', 'ROST',
                  'Position', 'Year']
    return df

In [9]:
def te_col(df):
    #tight end
    df.columns = ['Rank', 'Player', 'Receiving Receptions', 'Receiving Target', 'Receiving Yards',
                  'Yards/Reception', 'Longest Reception', 'Receiving 20+',
                  'Receiving Touchdowns', 'Rushing Attempts', 'Rushing Yards',
                  'Rushing Touchdown', 'Fumbles', 'Games Played', 'Fantasy Football Points',
                   'FTPS/Game', 'ROST', 'Position', 'Year']
    return df

In [10]:
def k_col(df):
    #kicking
    df.columns = ['Rank', 'Player', 'Field Goals', 'Field Attempts', 'Percentage',
                  'Longest Field', '1-19 Yards', '20-29 Yards', '30-39 Yards',
                  '40-49 Yards', '50+ Yards', 'Extra Points Made', 'Extra Points Attempted',
                  'Games Played', 'Fantasy Football Points',
                   'FTPS/Game', 'ROST', 'Position', 'Year']
    return df

In [11]:
def def_st_col(df):
    # Defese and Special Teams
    df.columns = ['Rank', 'Player', 'Sack', 'Interceptions', 
                  'Fumble Recovery', 'Forced Fumble', 'Defense Touchdown', 'Safety', 
                  'Special Touchdown', 'Games Played', 'Fantasy Football Points', 'FTPS/Game', 
                  'ROST', 'Position', 'Year']
    return df

In [12]:
def dl_col(df):
    # defensive lineman
    df.columns = ['Rank', 'Player', 'Tackle', 'Assist', 
                  'Sack', 'Pass Defended', 'Interception', 'Forced Fumble', 
                  'Fumble Recovery', 'Defend Touchdown', 'Games Played', 'Fantasy Football Points', 
                  'FTPS/Game', 'ROST', 'Position', 'Year']
    return df

In [13]:
def lb_col(df):
    #line backer
    df.columns = ['Rank', 'Player', 'Tackle', 'Assist', 'Sack', 'Pass Defended',
                  'Interception', 'Forced Fumble', 'Fumble Recovery', 'Defend Touchdown',
                  'Games Played', 'Fantasy Football Points', 'FTPS/Game', 'ROST', 'Position', 'Year']    
    return df

In [14]:
def db_col(df):
    # defensive back
    df.columns = ['Rank', 'Player', 'Tackle', 'Assist', 'Sack', 'Pass Defended',
                  'Interception', 'Forced Fumble', 'Fumble Recovery', 'Defend Touchdown',
                  'Games Played', 'Fantasy Football Points', 'FTPS/Game', 'ROST', 'Position', 'Year']
    return df

In [15]:
for year in years:    
    for week in weeks:
        for position in position_li:
            url = f'https://www.fantasypros.com/nfl/stats/{position}.php?week={week}&range=week&year={year}'

            df = pd.read_html(url)[0]

            time.sleep(random.randrange(7,10))

            df = clean_df(df, year, week, position)

            if position == 'qb': 
                df = qb_col(df)
            elif position == 'rb':
                df = rb_col(df)
            elif position == 'wr':
                df = wr_col(df)
            elif position == 'te':
                df = te_col(df)
            elif position == 'k':
                df = k_col(df)
            elif position == 'dst':
                df = def_st_col(df)
            elif position == 'dl':
                df = dl_col(df)
            elif position == 'lb':
                df = lb_col(df)
            elif position == 'db':
                df = db_col(df)
            else:
                print('DNE', position, year)


            df.to_csv(f'NFL_Data_{year}/Weekly Stats/{position}_{week}_{year}')

In [41]:
# Concat each position by year and upload to sql MSSQLSERVER2019
for position in position_li:

    position_df = pd.DataFrame()
    
    for year in years:

        file_li = os.listdir(f'NFL_Data_{year}/Weekly Stats/')

        for file in file_li:

            if position in file:
                df = pd.read_csv(f'NFL_Data_{year}/Weekly Stats/{file}', index_col=0)
                
                position_df = pd.concat([position_df, df], axis=0)


    position_df.to_sql(f'{position.upper()}_Player_Weekly_Stats', con=conn, if_exists='replace')


In [ ]:
# get team for each player data from sql, join TEAM column, save in SQL and CSV

for position in position_li:

    #join all years of that position
    query = f'''select {position}.*, Roster.Team
    from {position.upper()}_Player_Weekly_Stats {position}
    join Roster on
    ({position}.Player = Roster.Player) and
    ({position}.Year = Roster.Year);'''

    # read sql
    position_df = pd.read_sql(query, conn)

    
    position_df.to_sql(f'{position.upper()}_Player_Weekly_Stats', con=conn, if_exists='replace')
    position_df.to_csv(f'Weekly Stats/{position.upper()}_Player_Weekly_Stats')

In [ ]:
need to join date, day of the week... win/loss?

In [ ]:
stop

In [ ]:
url = 'https://www.fantasypros.com/nfl/stats/qb.php?week=1&range=week&year=2010'

In [ ]:
df = pd.read_html(url)[0]

In [ ]:
df = df.droplevel(0, axis=1)
df

In [ ]:
df['Player'] = df['Player'].apply(lambda x: x.split(' ')[:2])
df['Player'] = df['Player'].apply(lambda x: ' '.join(x))

In [ ]:
url = 'https://www.nfl.com/stats/player-stats/'

In [ ]:
pd.read_html(url)